In [1]:
import dgl
dgl.__version__

'0.9.0'

In [1]:
from functions_HGPLS import net_graph, train, test, extract_prediction
from graph_networks import HGPSLModel

import numpy as np
import pandas as pd
import scipy.sparse as sp

import dgl
import networkx as nx
import numpy as np
import torch
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm
from sklearn.decomposition import PCA

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("seyonec/SMILES_tokenized_PubChem_shard00_160k")

model = AutoModelForMaskedLM.from_pretrained("seyonec/SMILES_tokenized_PubChem_shard00_160k")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
atoms = ["", "H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm"]
print(len(atoms))
ids = tokenizer(atoms).input_ids

toks = list()
for i in ids:
    if len(i) >= 3:
        toks.append(i[1])
    else:
        toks.append(i[0])

100


In [4]:
dataset, labels_g = dgl.load_graphs("../data/HIV_dgl_graphs")

In [5]:
labels_g = labels_g['glabel'].tolist()

In [6]:
tok = torch.zeros((len(toks), 1))
for i in range(len(toks)):
    tok[i] = toks[i]

tok
transfo_atoms = model(tok.to(torch.long)).logits.reshape(100, -1)

In [7]:
pca = PCA(n_components=100, random_state=42)
res = pca.fit_transform(transfo_atoms.detach().numpy())

In [26]:
new_dataset = list()
for i in tqdm(range(len(dataset))):
    g= dataset[i]
    l = labels_g[i]
    ids=list(g.ndata["atomic"].numpy().flatten().astype(int))
    g.ndata["feature"] = torch.tensor(res[ids])
    g.edata["type"] = g.edata["type"].to(dtype=torch.float32)
    g = dgl.add_self_loop(g)
    g = dgl.add_reverse_edges(g, copy_edata=True)
    a=dgl.graph(g.edges(), idtype=torch.int64)
    a.ndata["feature"] = g.ndata["feature"]
    a.edata["type"] = g.edata["type"].flatten()
    new_dataset.append((a, l))

100%|██████████| 2443/2443 [00:02<00:00, 886.42it/s]


In [27]:
train_dataset, test_dataset = train_test_split(new_dataset, test_size=0.25, random_state=42)

train_dataloader = GraphDataLoader(train_dataset, batch_size=16, drop_last=False)
test_dataloader = GraphDataLoader(test_dataset, batch_size=16, drop_last=False)

In [28]:
# Load model architecture
device = 'cpu' if torch.cuda.is_available() else 'cpu'
model = HGPSLModel(in_feat=100, out_feat=3, hid_feat=256).to(device)

In [29]:
# Define optimizer and loss
optimizer = torch.optim.Adam(
        model.parameters(), lr=0.0001, #weight_decay=0.001
    )
loss = torch.nn.CrossEntropyLoss()

In [30]:
# Train model and keep the best validation loss model
bad_cound = 0
best_val_acc = 0
best_epoch = 0
epochs = 20
patience = 20
print_every = 1
train_times = []
for e in range(epochs):
    s_time = time()
    train_loss, train_acc = train(model, optimizer, loss, train_dataloader, device, True)
    train_times.append(time() - s_time)
    val_acc, val_loss = test(model, loss, test_dataloader, device, True)
    if best_val_acc < val_acc:
        best_val_acc = val_acc
        bad_cound = 0
        best_epoch = e + 1
        torch.save(model.state_dict(), "../models/HGPSLModel_prot.pt")
    else:
        bad_cound += 1
    if bad_cound >= patience:
        break

    if (e + 1) % print_every == 0:
        log_format = (
            "Epoch {}: train_loss={:.4f}, train_acc={:.4f}, val_acc={:.4f}, vall_loss={:.4f}"
        )
        print(log_format.format(e + 1, train_loss, train_acc, val_acc, val_loss))
print(
    "Best Epoch {}, final test acc {:.4f}".format(
        best_epoch, best_val_acc
    )
)

100%|██████████| 39/39 [00:05<00:00,  7.15it/s]


Epoch 1: train_loss=1.3761, train_acc=0.4154, val_acc=0.4403, vall_loss=1.0120


100%|██████████| 39/39 [00:05<00:00,  7.66it/s]


Epoch 2: train_loss=1.1276, train_acc=0.4061, val_acc=0.4845, vall_loss=1.0168


100%|██████████| 39/39 [00:05<00:00,  6.64it/s]


...

Epoch 16: train_loss=1.0249, train_acc=0.4672, val_acc=0.4894, vall_loss=0.9640


100%|██████████| 39/39 [00:07<00:00,  5.15it/s]


Epoch 17: train_loss=1.0088, train_acc=0.4793, val_acc=0.5106, vall_loss=0.9638


100%|██████████| 39/39 [00:05<00:00,  6.59it/s]

Best Epoch 8, final test acc 0.5155


### Load Best Model

In [31]:
model.load_state_dict(torch.load("../models/HGPSLModel_prot.pt"))

<All keys matched successfully>

### Extract last hidden layer

In [42]:
emb_test=[]
lab_test = []
for batch in tqdm(test_dataloader):
        batch_graphs, batch_labels = batch
        batch_graphs = batch_graphs.to(device)
        batch_labels = batch_labels.long().to(device)
        out = model(batch_graphs, batch_graphs.ndata["feature"].to(dtype=torch.float32))
        out=model.readout
        emb_test += out.tolist()
        lab_test += batch_labels.tolist()

100%|██████████| 39/39 [00:04<00:00,  8.01it/s]


In [43]:
emb_train=[]
lab_train = []
for batch in tqdm(train_dataloader):
        batch_graphs, batch_labels = batch
        batch_graphs = batch_graphs.to(device)
        batch_labels = batch_labels.long().to(device)
        out = model(batch_graphs, batch_graphs.ndata["feature"].to(dtype=torch.float32))
        out=model.readout
        emb_train += out.tolist()
        lab_train += batch_labels.tolist()

100%|██████████| 115/115 [00:16<00:00,  7.07it/s]


### XGBoost prediction

In [44]:
from xgboost import XGBClassifier
xgb =XGBClassifier(random_state=0)

xgb.fit(emb_train, lab_train)
predxgb = xgb.predict(emb_test)

In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
accuracy_score(lab_test, predxgb), f1_score(lab_test, predxgb, average="macro")

(0.6072013093289689, 0.6211857504339401)

In [46]:
confusion_matrix(lab_test, predxgb)

array([[161,  95,   5],
       [ 86, 150,  19],
       [  9,  26,  60]], dtype=int64)